In [1]:
import os
import requests
from dotenv import load_dotenv
import clickhouse_connect
import itertools
import pandas as pd
import json

load_dotenv()
clickhouse_host = os.getenv("CLICKHOUSE_HOST")
username = os.getenv("CLICKHOUSE_USER")
password = os.getenv("CLICKHOUSE_PASSWORD")
github_token = os.getenv("GITHUB_TOKEN")

client = clickhouse_connect.get_client(host=clickhouse_host, port=8123, username=username, password=password)

headers = {
  "Authorization": f"token {github_token}"
}

/Users/xiaoyawork/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 通用函数封装

In [2]:
def execute_query_relations(repo_id1, repo_id2, created_at='2024-01-01'):
  sql_query_relations = """ 
    -- 计算两个仓库之间的共同开发者数量
    WITH
      -- 获取第一个仓库在指定时间段内的活跃开发者
      active_developers_1 AS (
        SELECT DISTINCT actor_id
        FROM opensource.events
        WHERE repo_id = %s
        AND created_at >= '%s'
        AND (type IN ('IssuesEvent', 'PullRequestEvent', 'IssueCommentEvent','PullRequestReviewEvent','PullRequestReviewCommentEvent'))
      ),
      -- 获取第二个仓库在指定时间段内的活跃开发者
      active_developers_2 AS (
        SELECT DISTINCT actor_id
        FROM opensource.events
        WHERE repo_id = %s
        AND created_at >= '%s'
        AND (type IN ('IssuesEvent', 'PullRequestEvent', 'IssueCommentEvent', 'PullRequestReviewEvent', 'PullRequestReviewCommentEvent'))
      )
    -- 计算共同开发者数量
    SELECT COUNT(DISTINCT a.actor_id) AS common_developer_count
    FROM active_developers_1 a
    JOIN active_developers_2 b ON a.actor_id = b.actor_id
    """
  formatted_query = sql_query_relations % (f"'{repo_id1}'", created_at, f"'{repo_id2}'", created_at)
  results = client.query(formatted_query)
  return results

# 查询 github openrank top 项目列表
def execute_query_top_openrank(created_at='2025-01-01'):
  sql_query_top_openrank = """
    SELECT
        repo_id,
        repo_name,
        ROUND(AVG(openrank), 2) AS avg_openrank_25
    FROM
        opensource.global_openrank
    WHERE
        platform = 'GitHub' AND
        created_at >= %s
    GROUP BY
        repo_id, repo_name
    ORDER BY
        avg_openrank_25 DESC
    LIMIT 500
  """
  formatted_query = sql_query_top_openrank % (f"'{created_at}'")
  results = client.query(formatted_query)
  return results

# 根据 repo_id 查询仓库的 openrank 均值
def execute_nodes_avg_openrank(repo_id, created_at='2025-01-01'):
  sql_nodes_openrank = """
    SELECT repo_id, avg(openrank) AS average_openrank
    FROM opensource.global_openrank
    WHERE repo_id = %s
    AND platform = 'GitHub'
    AND created_at >= '%s'
    GROUP BY repo_id
  """
  formatted_query = sql_nodes_openrank % (f"'{repo_id}'", created_at)
  results = client.query(formatted_query)
  return results

# 根据 repo_id 仓库当月的 openrank 值
def execute_nodes_openrank(repo_id, created_at):
  sql_nodes_openrank = """
    SELECT repo_id, round(sum(openrank)) AS openrank
    FROM opensource.global_openrank
    WHERE repo_id = %s
    AND platform = 'GitHub'
    AND created_at = '%s'
    GROUP BY repo_id
  """
  formatted_query = sql_nodes_openrank % (f"'{repo_id}'", created_at)
  results = client.query(formatted_query)
  return results

# 根据 repo_id 查询最新的 repo_name
def execute_query_projects_name(repo_id):
    sql_query_project_name = """
        SELECT repo_id, repo_name
        FROM opensource.events
        WHERE repo_id IN (%s)
        AND platform = 'GitHub'
        ORDER BY created_at DESC
        LIMIT 1
        """
    formatted_query = sql_query_project_name % (f"'{repo_id}'")
    results = client.query(formatted_query)
    return results

# 根据 repo_name 查询最新的 repo_id
def execute_query_projects_id(repo_name):
    sql_query_project_name = """
      SELECT repo_id, repo_name
      FROM opensource.events
      WHERE repo_name = %s
      AND platform = 'GitHub'
      ORDER BY created_at DESC
      LIMIT 1
      """
    formatted_query = sql_query_project_name % (f"'{repo_name}'")
    results = client.query(formatted_query)
    return results
   
# 根据 repo_name, 通过 GitHub API 查询仓库基本信息
def get_repo_info(repo_name, headers):
    url = f"https://api.github.com/repos/{repo_name}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        repo_info = {
            "repo_id": data.get("id"),
            "repo_name": data.get("full_name"),
            "stargazers_count": data.get("stargazers_count"),
            "forks_count": data.get("forks_count"),
            "language": data.get("language"),
            "created_at": data.get("created_at").split("T")[0],
            "description": data.get("description"),
        }
        return repo_info
    else:
        print(f"Error fetching data for {repo_name}: {response.status_code}")
        return None

In [ ]:
df = pd.read_csv('landscapev2505_full.csv', encoding='latin1')
repo_ids = df['repo_id'].tolist()
repo_names = df['repo_name'].tolist()

repo_names = set(repo_names)
len(repo_names)

## 获取（更新） Landscape 项目列表的基本信息

In [3]:
import pandas as pd
from datetime import datetime
import json

# Format repositories as a list of strings
repo_names = [
  "NixOS/nixpkgs",
  "llvm/llvm-project",
  "home-assistant/core",
  "pytorch/pytorch",
  "odoo/odoo",
  "zephyrproject-rtos/zephyr",
  "microsoft/vscode",
  "microsoft/winget-pkgs",
  "elastic/kibana",
  "DigitalPlatDev/FreeDomain",
  "vllm-project/vllm",
  "godotengine/godot",
  "flutter/flutter",
  "Expensify/App",
  "zed-industries/zed",
  "department-of-veterans-affairs/va.gov-team",
  "grafana/grafana",
  "rust-lang/rust",
  "langgenius/dify",
  "digitalinnovationone/dio-lab-open-source",
  "openjdk/jdk",
  "WebKit/WebKit",
  "is-a-dev/register",
  "Kas-tle/java2bedrock.sh",
  "Automattic/wp-calypso",
  "ClickHouse/ClickHouse",
  "dotnet/runtime",
  "openshift/openshift-docs",
  "python/cpython",
  "openshift/release",
  "getsentry/sentry",
  "leanprover-community/mathlib4",
  "elastic/elasticsearch",
  "tenstorrent/tt-metal",
  "firstcontributions/first-contributions",
  "oven-sh/bun",
  "zen-browser/desktop",
  "DataDog/datadog-agent",
  "expo/expo",
  "huggingface/transformers",
  "vercel/next.js",
  "JetBrains/swot",
  "microsoft/vscode-copilot-release",
  "kubernetes/kubernetes",
  "ollama/ollama",
  "CherryHQ/cherry-studio",
  "woocommerce/woocommerce",
  "sgl-project/sglang",
  "raycast/extensions",
  "apache/doris",
  "ceph/ceph",
  "apache/airflow",
  "openvinotoolkit/openvino",
  "astral-sh/uv",
  "space-wizards/space-station-14",
  "CypherV2/Donarev-API",
  "DataDog/documentation",
  "ydb-platform/ydb",
  "BerriAI/litellm",
  "AliceO2Group/O2Physics",
  "open-webui/open-webui",
  "Azure/azure-rest-api-specs",
  "metabase/metabase",
  "department-of-veterans-affairs/vets-website",
  "FreeCAD/FreeCAD",
  "cloudflare/cloudflare-docs",
  "tgstation/tgstation",
  "openwrt/openwrt",
  "ultralytics/ultralytics",
  "cilium/cilium",
  "ray-project/ray",
  "dotnet/maui",
  "demisto/content",
  "nrfconnect/sdk-nrf",
  "n8n-io/n8n",
  "Koenkk/zigbee2mqtt",
  "openjournals/joss-reviews",
  "department-of-veterans-affairs/vets-api",
  "intel/llvm",
  "microsoft/PowerToys",
  "PaddlePaddle/Paddle",
  "bitwarden/clients",
  "swiftlang/swift",
  "Automattic/jetpack",
  "cockroachdb/cockroach",
  "StarRocks/starrocks",
  "infiniflow/ragflow",
  "immich-app/immich",
  "facebook/react-native",
  "stackblitz/bolt.new",
  "DaoCloud/public-image-mirror",
  "ibm-developer-skills-network/jbbmo-Introduction-to-Git-and-GitHub",
  "sonic-net/sonic-mgmt",
  "microsoft/vcpkg",
  "keycloak/keycloak",
  "airbytehq/airbyte",
  "pytorch/executorch",
  "Homebrew/homebrew-core",
  "NVIDIA/NeMo",
  "golang/go",
  "gravitational/teleport",
  "microsoft/onnxruntime",
  "apache/spark",
  "ggml-org/llama.cpp",
  "ArweaveOasis/Arweave-Academy",
  "bevyengine/bevy",
  "CleverRaven/Cataclysm-DDA",
  "supabase/supabase",
  "Homebrew/homebrew-cask",
  "open-telemetry/opentelemetry-collector-contrib",
  "project-chip/connectedhomeip",
  "WordPress/gutenberg",
  "gentoo/gentoo",
  "camunda/camunda",
  "PostHog/posthog",
  "apache/kafka",
  "void-linux/void-packages",
  "spack/spack",
  "SoftFever/OrcaSlicer",
  "elizaOS/eliza",
  "calcom/cal.com",
  "dagster-io/dagster",
  "backstage/backstage",
  "tailscale/tailscale",
  "payloadcms/payload",
  "MetaMask/metamask-extension",
  "continuedev/continue",
  "shopware/shopware",
  "DefiLlama/DefiLlama-Adapters",
  "evcc-io/evcc",
  "angular/angular",
  "lobehub/lobe-chat",
  "langflow-ai/langflow",
  "astral-sh/ruff",
  "systemd/systemd",
  "brave/brave-core",
  "zulip/zulip",
  "smartcontractkit/chainlink",
  "yugabyte/yugabyte-db",
  "fleetdm/fleet",
  "neovim/neovim",
  "envoyproxy/envoy",
  "argoproj/argo-cd",
  "hashicorp/terraform-provider-aws",
  "quarkusio/quarkus",
  "comfyanonymous/ComfyUI",
  "justachillcoder/binance-captcha-deobfuscator",
  "milvus-io/milvus",
  "kubernetes/website",
  "nextcloud/server",
  "pingcap/tidb",
  "wolfi-dev/os",
  "appsmithorg/appsmith",
  "keiyoushi/extensions-source",
  "langchain-ai/langchain",
  "scylladb/scylladb",
  "cline/cline",
  "yt-dlp/yt-dlp",
  "Azure/azure-sdk-for-python",
  "paritytech/polkadot-sdk",
  "hyprwm/Hyprland",
  "pola-rs/polars",
  "ziglang/zig",
  "home-assistant/home-assistant.io",
  "hashicorp/terraform-provider-azurerm",
  "nrwl/nx",
  "nodejs/node",
  "apache/superset",
  "espressif/esp-idf",
  "mdn/content",
  "GoogleCloudPlatform/magic-modules",
  "trinodb/trino",
  "freeCodeCamp/freeCodeCamp",
  "medusajs/medusa",
  "bluesky-social/social-app",
  "SXueLing/BadouNLP",
  "better-auth/better-auth",
  "Azure/azure-sdk-for-net",
  "laravel/framework",
  "AdguardTeam/AdguardFilters",
  "dotnet/aspire",
  "musescore/MuseScore",
  "apache/datafusion",
  "deckhouse/deckhouse",
  "aws/aws-cdk",
  "huggingface/diffusers",
  "wazuh/wazuh",
  "brave/brave-browser",
  "harness/developer-hub",
  "denoland/deno",
  "cms-sw/cmssw",
  "shadcn-ui/ui",
  "microsoft/playwright",
  "hiyouga/LLaMA-Factory",
  "AztecProtocol/aztec-packages",
  "MystenLabs/sui",
  "openssl/openssl",
  "facebookincubator/velox",
  "bioconda/bioconda-recipes",
  "apache/iceberg",
  "rairprotocol/dev-dapp-season1",
  "microsoft/azurelinux",
  "gradle/gradle",
  "sonic-net/sonic-buildimage",
  "Dolibarr/dolibarr",
  "All-Hands-AI/OpenHands",
  "ant-design/ant-design",
  "macports/macports-ports",
  "MetaMask/metamask-mobile",
  "elastic/integrations",
  "symfony/symfony",
  "MicrosoftDocs/azure-docs",
  "dbeaver/dbeaver",
  "SAP/ui5-webcomponents",
  "LeetCode-Feedback/LeetCode-Feedback",
  "dotnet/roslyn",
  "juspay/hyperswitch",
  "home-assistant/frontend",
  "mattermost/mattermost",
  "strapi/strapi",
  "Flowseal/zapret-discord-youtube",
  "go-gitea/gitea",
  "toeverything/AFFiNE",
  "dotnet/sdk",
  "hashicorp/vault",
  "code-dot-org/code-dot-org",
  "LadybirdBrowser/ladybird",
  "ParadiseSS13/Paradise",
  "os-autoinst/os-autoinst-distri-opensuse",
  "twentyhq/twenty",
  "grafana/loki",
  "trezor/trezor-suite",
  "bambulab/BambuStudio",
  "NVIDIA/TensorRT-LLM",
  "bitcoin/bitcoin",
  "open-metadata/OpenMetadata",
  "opensearch-project/OpenSearch",
  "dotnet/aspnetcore",
  "run-llama/llama_index",
  "electron/electron",
  "cmss13-devs/cmss13",
  "Aider-AI/aider",
  "rails/rails",
  "dataease/dataease",
  "vercel/ai",
  "mui/mui-x",
  "nammayatri/nammayatri",
  "dfinity/ic",
  "triton-lang/triton",
  "libsdl-org/SDL",
  "uBlockOrigin/uAssets",
  "ovh/manager",
  "cloudflare/workers-sdk",
  "tonkeeper/ton-assets",
  "LedgerHQ/ledger-live",
  "YoYoGames/GameMaker-Bugs",
  "modelscope/ms-swift",
  "Azure/azure-cli",
  "paradigmxyz/reth",
  "obsidianmd/obsidian-releases",
  "qgis/QGIS",
  "ohcnetwork/care_fe",
  "apache/nuttx",
  "unslothai/unsloth",
  "istio/istio",
  "facebook/react",
  "stackrox/stackrox",
  "rancher/rancher",
  "OpenRailAssociation/osrd",
  "island-is/island.is",
  "starkware-libs/sequencer",
  "kubevirt/kubevirt",
  "sveltejs/svelte",
  "prestodb/presto",
  "sourcegraph/cody",
  "Facepunch/sbox-issues",
  "bcgov/entity",
  "mozilla-mobile/firefox-ios",
  "aptos-labs/aptos-core",
  "nuxt/nuxt",
  "umbraco/Umbraco-CMS",
  "OpenLiberty/open-liberty",
  "nikgapps/config",
  "RocketChat/Rocket.Chat",
  "clash-verge-rev/clash-verge-rev",
  "MetaMask/eth-phishing-detect",
  "termux/termux-packages",
  "github/codeql",
  "Azure/azure-sdk-for-js",
  "coollabsio/coolify",
  "grafana/mimir",
  "microsoft/FluidFramework",
  "elementor/elementor",
  "webcompat/web-bugs",
  "taosdata/TDengine",
  "microsoft/semantic-kernel",
  "volcengine/verl",
  "mdn/translated-content",
  "backstage/community-plugins",
  "Azure/azure-sdk-for-java",
  "thingsboard/thingsboard",
  "microsoft/fluentui",
  "JuliaLang/julia",
  "trustwallet/assets",
  "apache/pinot",
  "frappe/erpnext",
  "Bubberstation/Bubberstation",
  "TravelMapping/UserData",
  "ValveSoftware/Dota2-Gameplay",
  "tailwindlabs/tailwindcss",
  "AZPixel-Team/Java2Bedrock",
  "OpenCTI-Platform/opencti",
  "anza-xyz/agave",
  "apache/beam",
  "bitnami/charts",
  "apache/flink",
  "Koenkk/zigbee-herdsman-converters",
  "duckdb/duckdb",
  "streamlit/streamlit",
  "php/php-src"
]

results = []

for repo_name in repo_names:
  print(f"Processing {repo_name}...")
  
  # Get repo info from GitHub API
  repo_info = get_repo_info(repo_name, headers)
  
  if repo_info:
    repo_id = repo_info["repo_id"]
    
    openrank_result = execute_nodes_avg_openrank(repo_id, '2025-01-01')
    openrank_25 = openrank_result.result_rows[0][1] if openrank_result.result_rows else None
    
    # Compile all information
    result = {
      "repo_id": repo_id,
      "repo_name": repo_info["repo_name"],
      "stars": repo_info["stargazers_count"],
      "forks": repo_info["forks_count"],
      "openrank_25": round(openrank_25) if openrank_25 else None,
      "language": repo_info["language"],
      "created_at": repo_info["created_at"].split("T")[0],
      "description": repo_info["description"]
    }
    
    results.append(result)
  else:
    print(f"Failed to get info for {repo_name}")

df_results = pd.DataFrame(results)
columns_order = ["repo_id", "repo_name", "stars", "forks", "openrank_25", "language", "created_at", "description"]
df_results = df_results[columns_order]

csv_filename = f"repo_info_with_openrank_{datetime.now().strftime('%Y%m%d')}.csv"
df_results.to_csv(csv_filename, index=False)

print(f"Successfully processed {len(results)} repositories")
print(f"Results saved to {csv_filename}")

Processing NixOS/nixpkgs...
Processing llvm/llvm-project...
Processing home-assistant/core...
Processing pytorch/pytorch...
Processing odoo/odoo...
Processing zephyrproject-rtos/zephyr...
Processing microsoft/vscode...
Processing microsoft/winget-pkgs...
Processing elastic/kibana...
Processing DigitalPlatDev/FreeDomain...
Processing vllm-project/vllm...
Processing godotengine/godot...
Processing flutter/flutter...
Processing Expensify/App...
Processing zed-industries/zed...
Processing department-of-veterans-affairs/va.gov-team...
Processing grafana/grafana...
Processing rust-lang/rust...
Processing langgenius/dify...
Processing digitalinnovationone/dio-lab-open-source...
Processing openjdk/jdk...
Processing WebKit/WebKit...
Processing is-a-dev/register...
Processing Kas-tle/java2bedrock.sh...
Processing Automattic/wp-calypso...
Processing ClickHouse/ClickHouse...
Processing dotnet/runtime...
Processing openshift/openshift-docs...
Processing python/cpython...
Processing openshift/releas

## 生成 Landscape 项目关联网络

In [ ]:
# Initialize landscape_projects dictionary
landscape_projects = {}

# Prepare the data structure
for i, repo_id in enumerate(repo_ids):
    repo_name = df.loc[df['repo_id'] == repo_id, 'repo_name'].values[0]
    landscape_projects[repo_id] = {"name": repo_name}

# Query openrank for each repo and filter
# for repo_id in list(landscape_projects.keys()):
#     openrank = execute_nodes_avg_openrank(repo_id)
#     if openrank and openrank.result_rows:
#         print(openrank.result_rows)
#         average_openrank = int(openrank.result_rows[0][1])
#         landscape_projects[repo_id]["openrank"] = average_openrank
#         if average_openrank < 10:
#             del landscape_projects[repo_id]
#     else:
#         del landscape_projects[repo_id]



# Use the OpenRank values already available in the dataframe
for repo_id in list(landscape_projects.keys()):
  if repo_id in df['repo_id'].values:
    openrank = df.loc[df['repo_id'] == repo_id, 'openrank_25'].values[0]
    landscape_projects[repo_id]["openrank"] = openrank  
  else:
    del landscape_projects[repo_id]


nodes = []
for repo_id, info in landscape_projects.items():
  nodes.append([info["name"], info["openrank"]])

edges = []
for key1, key2 in itertools.combinations(landscape_projects.keys(), 2):
    project1 = landscape_projects[key1]['name']
    project2 = landscape_projects[key2]['name']

    query_result = execute_query_relations(key1, key2)
    if query_result.result_rows:
        value = query_result.result_rows[0][0]
        edges.append([project1, project2, value])

filtered_edges = [edge for edge in edges if edge[2] >= 10]

print(landscape_projects)
print(len(nodes),len(filtered_edges))

In [ ]:
filtered_edges = [filtered_edges for filtered_edges in filtered_edges if filtered_edges[2] >= 20]
print(len(filtered_edges))

In [ ]:
# 将 nodes 和 edges 组合并存储到 JSON 文件, 保存到 graph.json 文件中
# Convert numpy.int64 values to Python int for JSON serialization
nodes_converted = [[name, int(rank)] for name, rank in nodes]
edges_converted = [[source, target, int(value)] for source, target, value in filtered_edges]

graph = {
    "nodes": nodes_converted,
    "edges": edges_converted
}

# with open('./graph_landscape_202505.json', 'w') as json_file:
with open('graph_landscape_202505.json', 'w') as json_file:
  json.dump(graph, json_file, indent=4)


## 一、整体数据分析

### 1.1 获取 24-04 至 25-04 OpenRank 值，计算增幅

In [ ]:
# Get OpenRank values for January 2025 and April 2025
results = []

for repo_id in repo_ids:
  repo_name = df.loc[df['repo_id'] == repo_id, 'repo_name'].values[0] if repo_id in df['repo_id'].values else "Unknown"
  
  # Get start OpenRank
  start_openrank_res = execute_nodes_openrank(repo_id, '2024-04-01')
  start_openrank = start_openrank_res.result_rows[0][1] if start_openrank_res.result_rows else None
  
  # Get end OpenRank
  end_openrank_res = execute_nodes_openrank(repo_id, '2025-04-01')
  end_openrank = end_openrank_res.result_rows[0][1] if end_openrank_res.result_rows else None
  
  # Calculate growth metrics if both values exist
  if start_openrank is not None and end_openrank is not None:
    growth_value = end_openrank - start_openrank
    growth_ratio = (growth_value / start_openrank * 100) if start_openrank > 0 else float('inf')
    
    results.append({
      'repo_id': repo_id,
      'repo_name': repo_name,
      'start_openrank': start_openrank,
      'end_openrank': end_openrank,
      'growth_value': growth_value,
      'growth_ratio': growth_ratio
    })
    
    print(f"Processed {repo_name}: Start={start_openrank},  End={end_openrank}, Growth={growth_value}, Ratio={growth_ratio:.2f}%")
  else:
    print(f"Skipped {repo_name}: Missing OpenRank data")

# Convert to DataFrame and save as CSV
growth_df = pd.DataFrame(results)
growth_df = growth_df.sort_values(by='growth_ratio', ascending=False)
growth_df.to_csv('openrank_growth.csv', index=False)

print(f"Processed {len(results)} repositories with complete data")
print(f"Results saved to openrank_growth.csv")

### 1.2 计算两个项目的参与者人数和参与者重合

In [ ]:
def execute_query_community_openrank(repo_id):
  sql_query = """
  SELECT
      actor_login AS developer_name,
      ROUND(SUM(openrank), 2) AS total_openrank
  FROM
      opensource.community_openrank
  WHERE
      repo_id = %s
  GROUP BY
      actor_login
  ORDER BY
      total_openrank DESC
  """
  formatted_query = sql_query % (f"'{repo_id}'")
  results = client.query(formatted_query)
  return results

# Get developers for Dify
dify_repo_id = '626805178'
dify_developers = execute_query_community_openrank(dify_repo_id)
dify_dev_df = pd.DataFrame(dify_developers.result_rows, columns=['developer_name', 'total_openrank'])
print(f"Dify has {len(dify_dev_df)} developers")

# Get developers for LangChain
langchain_repo_id = '552661142'
langchain_developers = execute_query_community_openrank(langchain_repo_id)
langchain_dev_df = pd.DataFrame(langchain_developers.result_rows, columns=['developer_name', 'total_openrank'])
print(f"LangChain has {len(langchain_dev_df)} developers")

# Find common developers
common_developers = pd.merge(dify_dev_df, langchain_dev_df, 
                            on='developer_name', 
                            suffixes=('_dify', '_langchain'))
print(f"Number of common developers: {len(common_developers)}")

# Find common developers in OpenRank top 100 for both projects
dify_top100 = dify_dev_df.nlargest(500, 'total_openrank')
langchain_top100 = langchain_dev_df.nlargest(500, 'total_openrank')
common_top100 = pd.merge(dify_top100, langchain_top100, 
                         on='developer_name', 
                         suffixes=('_dify', '_langchain'))
print(f"Number of common developers in OpenRank top 100: {len(common_top100)}")

# Display the common developers
print("\nCommon developers in top 100:")
print(common_top100[['developer_name', 'total_openrank_dify', 'total_openrank_langchain']])